## Prediction

This code provides a command-line interface for predicting recovery time based on exercise and sleep metrics. It begins by loading a pre-trained model from a file (`svr_model.pkl`) and defining a list of required features, including exercise duration, steps, calories, heart rate, and sleep efficiency. The `standardize()` function scales each feature using pre-defined training means and standard deviations to ensure consistency with the model's expectations. 

The main function, `predict_recovery()`, prompts the user to enter values for each feature, converting durations from minutes to milliseconds where necessary. It then standardizes the inputs and generates a recovery time prediction. The prediction is classified as faster, average, or slower recovery based on predefined thresholds. The code continuously runs this prediction loop until the user opts to exit.


In [1]:
import joblib
import numpy as np
import warnings
from sklearn.exceptions import DataConversionWarning


# Load the pre-trained model
model = joblib.load('../3.ModelSelectionAndTraining/svr_model.pkl')

# Feature names in the correct order
features = [
    'exercise_activeDuration_in_min',
    'exercise_steps',
    'exercise_calories',
    'exercise_elevationGain_in_m',
    'exercise_averageHeartRate',
    'sleep_duration_in_min',
    'sleep_minutesAsleep',
    'sleep_minutesAwake',
    'sleep_efficiency'
]


def standardize(values):
    TRAINING_MEAN = [4410746.42, 6234.74, 637.47, 109.07, 114.49, 29467008.0, 428.43, 62.32, 94.56]
    TRAINING_STD = [3991314.52, 4589.22, 479.17, 172.94, 18.41, 7647729.37, 109.3, 25.17, 4.99]
    return np.array([(v - m)/s for v,m,s in zip(values, TRAINING_MEAN, TRAINING_STD)]).reshape(1, -1)

def predict_recovery():
    print("Enter values for each feature (press Enter after each):")
    
    # Collect raw inputs
    raw_values = []
    for feature in features:
        while True:
            try:
                if feature == "exercise_activeDuration_in_min" or feature == "sleep_duration_in_min":   
                    value = float(input(f"{feature}: "))
                    value = value * 60 * 1000 # Convert to ms
                else:
                    value = float(input(f"{feature}: "))
                    
                raw_values.append(value)
                break
            except ValueError:
                print("Please enter a valid number")
    
    # standardize
    scaled_data = standardize(raw_values)
    
    # Make prediction
    prediction = model.predict(scaled_data)[0]
    
    print(f"\nPredicted Recovery Time: {prediction:.2f} hours")
    
    
    # Simple interpretation
    if prediction < 18:
        print("(Faster than average recovery)")
    elif prediction > 25:
        print("(Slower than average recovery)")
    else:
        print("(Average recovery time)")

warnings.filterwarnings("ignore", category=UserWarning)

# Run Predictions
while True:
    predict_recovery()
    print("\n" + "="*50 + "\n")
    if input("Make another prediction? (y/n): ").lower() != 'y':
        break

Enter values for each feature (press Enter after each):


exercise_activeDuration_in_min:  41
exercise_steps:  3035
exercise_calories:  354
exercise_elevationGain_in_m:  21.34
exercise_averageHeartRate:  98
sleep_duration_in_min:  430
sleep_minutesAsleep:  378
sleep_minutesAwake:  52
sleep_efficiency:  97



Predicted Recovery Time: 20.65 hours
(Average recovery time)




Make another prediction? (y/n):  n
